In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import cv2
import numpy as np
from sklearn.cluster import KMeans
from deepface import DeepFace
from tkinter import *
from tkinter import filedialog

# ------------------ FairFace Model Setup ------------------
# Load FairFace model for ethnicity/nationality
model = torch.hub.load('joojs/fairface', 'fairface_race')
model.eval()

# Preprocessing for FairFace
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

race_classes = ['White', 'Black', 'Indian', 'East Asian', 'Southeast Asian', 'Middle Eastern', 'Latino_Hispanic']

In [ ]:
def predict_nationality(image_path):
    img = Image.open(image_path).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0)
    with torch.no_grad():
        outputs = model(img_tensor)
        _, pred = torch.max(outputs, 1)
    return race_classes[pred.item()]

# ------------------ DeepFace Emotion & Age ------------------
def predict_emotion_and_age(image_path):
    try:
        analysis = DeepFace.analyze(image_path, actions=['age','emotion'], enforce_detection=False)
        emotion = analysis[0]['dominant_emotion']
        age = analysis[0]['age']
        return emotion, age
    except:
        return "Unknown", "Unknown"

# ------------------ Dress Color Detection ------------------
def predict_dress_color(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return "Unknown"
    # Crop lower part of the face to approximate clothing region
    height, width, _ = img.shape
    crop_img = img[int(height*0.5):, :]  # bottom half
    crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
    pixels = crop_img.reshape(-1, 3)
    kmeans = KMeans(n_clusters=1, random_state=0).fit(pixels)
    dominant_color = kmeans.cluster_centers_[0].astype(int)
    return f"RGB({dominant_color[0]}, {dominant_color[1]}, {dominant_color[2]})"

# ------------------ Tkinter GUI ------------------
root = Tk()
root.title("Nationality & Emotion Detection")
root.geometry("900x600")


In [ ]:
def open_image():
    file_path = filedialog.askopenfilename()
    if not file_path:
        return

    # Display the image
    img = Image.open(file_path)
    img = img.resize((250, 250))
    img_tk = ImageTk.PhotoImage(img)
    panel.config(image=img_tk)
    panel.image = img_tk

    # Predictions
    nationality = predict_nationality(file_path)
    emotion, age = predict_emotion_and_age(file_path)
    dress_color = predict_dress_color(file_path)

    # Determine what to show based on nationality
    result_text = f"Nationality: {nationality}\nEmotion: {emotion}"
    if nationality == "Indian":
        result_text += f"\nAge: {age}\nDress Color: {dress_color}"
    elif nationality == "White":  # assuming White represents US
        result_text += f"\nAge: {age}"
    elif nationality == "Black":  # assuming African
        result_text += f"\nDress Color: {dress_color}"
    # Other nationalities: only show nationality + emotion (already included)

    result_label.config(text=result_text)

# GUI Widgets
open_button = Button(root, text="Open Image", command=open_image, font=("Arial", 14))
open_button.pack(pady=10)

panel = Label(root)
panel.pack(pady=10)

result_label = Label(root, text="", font=("Arial", 14), justify=LEFT)
result_label.pack(pady=10)

root.mainloop()
